In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
from delierium.helpers import is_derivative, order_of_derivative
from delierium.MatrixOrder import Context
import functools
from operator import mul
from pprint import pprint

In [2]:
x = var('x')
y = function("y")
from IPython.core.debugger import set_trace
function('xi phi')
def order_of_ode(ode, dependent, independent):
    # XXX write tests
    m = 0
    if is_derivative(ode):
        m = max(order_of_derivative(ode)[0], m)
    elif ode.operator().__name__ == 'mul_vararg':
        for i in ode.operands():
            if is_derivative(i):
                m = max(m, order_of_derivative(i)[0])
    elif ode.operator().__name__ == 'add_vararg':
        for i in ode.operands():
            if is_derivative(i):
                m = max(m, order_of_derivative(i)[0])
            else:
                for j in i.operands():
                    if is_derivative(j):
                        m = max(m, order_of_derivative(j)[0])
    return m
    
def infinitesimalsODE (ode, dependent, independent):
    order = order_of_ode (ode, dependent, independent)
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    # XXX: get correct order of ode
    s1                 = solve(ode==0, diff(dependent(independent),independent, order))
    determining_system = prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify().expand()
    # so far it is checked manually and with mathematica
    #set_trace()
    all_derivative_monomials = set()
    for term in determining_system.operands():
        collector = 1
        for o in term.operands():
            if hasattr(o.operator(), "__name__") and (o.operator().__name__ == 'pow'):
                base, exponent = o.operands()
                if is_derivative(base) and base.operator().function() == dependent:
                    collector *= o
            elif is_derivative(o) and o.operator().function() == dependent:
                collector *= o
            else:
                pass
        if collector != 1:
            all_derivative_monomials.add(collector)
    class mon:
        # helper class just
        def __init__ (self, it):
            self._it = it
        def __lt__ (self, other):
            if is_derivative(self._it) and is_derivative(other._it):
                return order_of_derivative(self._it)[0] < order_of_derivative(other._it)[0]
            elif is_derivative(self._it) and other._it.operator().__name__ in ['mul_vararg', 'pow']:
                return True
            elif is_derivative(other._it) and self._it.operator().__name__ in ['mul_vararg', 'pow']:
                return False
            elif self._it.operator().__name__ == 'pow' == other._it.operator().__name__ :
                if self._it.operands()[0] == other._it.operands()[0]:
                    return self._it.operands()[1] < other._it.operands()[1]
                else:                
                    return self._it.operands()[1] < other._it.operands()[1]
            elif self._it.operator().__name__ == 'pow' and other._it.operator().__name__ == 'mul_vararg':
                return True
            elif self._it.operator().__name__ == 'mul_vararg' and other._it.operator().__name__ == 'pow':
                return False
            elif 'mul_vararg' == self._it.operator().__name__ == other._it.operator().__name__ :
                if len(self._it.operands()) < len(other._it.operands()):
                    return False
                elif len(self._it.operands()) > len(other._it.operands()):
                    return True
                else:
                    s = self._it.operands()
                    o = other._it.operands()
                    s.sort()
                    o.sort()
                    for _s, _o in zip(s, o):
                        if is_derivative(_s) and is_derivative(_o):
                            if order_of_derivative(_s)[0] < order_of_derivative(_o)[0]:
                                return True
                            else:
                                return False
                        elif is_derivative(_s) and not is_derivative(_o):
                            return True
                        elif is_derivative(_o) and not is_derivative(_s):
                            return False
                        elif _s.operator().__name__ == 'pow' == _o.operator().__name__:
                            left  = _s.operands()
                            right = _o.operands()
                            set_trace()
                            pass
                        else:
                            set_trace()
                            pass
            else:
                print("Shithole")
                return True
    derivative_monomials = [_ for _ in reversed(sorted([mon(_) for _ in all_derivative_monomials]))]
    _det = []
    for monomial in derivative_monomials:
        monomial = monomial._it
        if is_derivative(monomial) or monomial.operator().__name__ == 'pow':
            _v = set([monomial])
        else:
            _v = set(monomial.operands())
        coeffsum = 0
        for _coll in determining_system.operands():
            if _coll == monomial:
                coeffsum += 1
            else:
                factors = set(_coll.operands())
                if _v.issubset(factors):
                    coeffsum += functools.reduce(mul, factors - _v, 1)
        if coeffsum:
            _det.append(coeffsum)
        determining_system = (determining_system - coeffsum * monomial).expand()
        
    # append remaining terms which have no derivatives ("the constant term")
    _det.append(determining_system)
    determining_system = _det
    print ("this is my determining system")
    for _ in determining_system:
        _.show()
    
    #set_trace()
    X, Y = var('X Y')
    for i in range(len(determining_system)):
        determining_system[i] = determining_system[i].subs({dependent(independent) : Y})
        determining_system[i] = determining_system[i].subs({diff(dependent(independent), independent) : 0})
    # toDo: repair ASAP
    janet = Janet_Basis(determining_system, [xi, phi], [Y, independent])
    pols = []
    for i in range(len(janet.S)):
        pols.append(janet.S[i].expression().subs({Y : dependent(independent)}))
    return pols

In [3]:
# Schwarz, Example 5.2, p. 200
x = var('x')
y = function("y")
ode = 4*diff(y(x),x,x)*y(x) - 3*diff(y(x), x)**2-12*y(x)**3
inf = infinitesimalsODE(ode, y, x)
print("."*25)
for _ in inf:
    _.show()

/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/expression_conversions.py:2347: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self.new(*[self(_) for _ in ex.operands()])
/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/operators.py:121: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self._f(*args).diff(*vars)


this is my determining system


-4*y(x)*D[0, 0](xi)(y(x), x) - 3*D[0](xi)(y(x), x)

4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) + 3*phi(y(x), x)/y(x) - 3*D[0](phi)(y(x), x)

-36*y(x)^3*D[0](xi)(y(x), x) + 8*y(x)*D[0, 1](phi)(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x) - 6*D[1](phi)(y(x), x)

...

.........................


D[1](phi)(y(x), x)

-phi(y(x), x)/y(x) + D[0](phi)(y(x), x)

1/2*phi(y(x), x)/y(x) + D[1](xi)(y(x), x)

D[0](xi)(y(x), x)

In [4]:
# Schwarz, Example 5.9, p.214",
x = var('x')
y = function("y")
ode = 4*diff(y(x),x,x)*x**2 + (x**4) * diff(y(x), x)**2 + 4*y(x)
inf = infinitesimalsODE(ode, y, x)
print("."*25)
for _ in inf:
    _.show()

/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/operators.py:121: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self._f(*args).diff(*vars)
/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/expression_conversions.py:2347: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self.new(*[self(_) for _ in ex.operands()])


this is my determining system


x^4*D[0](xi)(y(x), x) - 4*x^2*D[0, 0](xi)(y(x), x)

x^4*D[0](phi)(y(x), x) + 2*x^3*xi(y(x), x) + 4*x^2*D[0, 0](phi)(y(x), x) - 8*x^2*D[0, 1](xi)(y(x), x)

2*x^4*D[1](phi)(y(x), x) + 8*x^2*D[0, 1](phi)(y(x), x) - 4*x^2*D[1, 1](xi)(y(x), x) + 12*y(x)*D[0](xi)(y(x), x)

...

diff(xi(Y, x), Y, Y)

diff(phi(Y, x), Y, x, x)

diff(phi(Y, x), Y, Y, x)

diff(phi(Y, x), Y, Y, Y)

ValueError: tuple.index(x): x not in tuple

In [ ]:
# Schwarz, Example 5.11, p 215
ode = (x-diff(y(x), x))*diff(y(x), x,x) - 4 * diff(y(x), x)**2
inf = infinitesimalsODE(ode, y, x)
print("."*25)
for _ in inf:
    _.show()

In [ ]:
for _ in inf:
    _.show()

In [ ]:
# Arrigo, example 2.17
ode = diff(y(x), x,x)
inf = infinitesimalsODE(ode, y, x)
print("========")
for _ in inf:
    _.show()

In [ ]:
ode2=4*diff(y(x),x,x)*y(x)**2 - 3* (x**2)*diff(y(x),x)*2-12*y(x)**3
inf = infinitesimalsODE(ode2, y ,x)
print("=======")
for _ in inf:
    _.show()

In [ ]:
ode3=diff(y(x),x,x) + y(x)*diff(y(x),x)+x*y(x)**4
inf = infinitesimalsODE(ode3, y ,x)
print("=========")
for _ in inf:
    _.show()

In [ ]:
            print ("A"*99)
            print (potential_funcs[0], zip(potential_funcs[0], e2_order))
#xi = function('xi')
#phi = function('phi')
#x, y = var("x,y")#
#ctx = Context([xi, phi], [y,x])
#m = [-diff(xi(y,x), y,y),
#                  2*y*diff(xi(y,x), y) + diff(phi(y,x), y,y) - 2*diff(xi(y,x), x,y),
#                  3*x*(y**4)*diff(xi(y,x), y) + y*diff(xi(x,y),x) + phi(y,x) + 2*diff(phi(y,x),x) - diff(xi(y,x),x,x),
#                  -x*(y**4)*diff(phi(y,x),y) + 2*x*(y**4)*diff(xi(y,x), x)+4*x*phi(y,x) * (y**3) + xi(y,x)*(y**4)+y*diff(phi(y,x),x)+diff(phi(y,x),x,x) 
#                 ]            print ("A"*99)
            print (potential_funcs[0], zip(potential_funcs[0], e2_order))

#for _ in m:
#    _.show()
#jj = Janet_Basis(m, ctx._dependent, ctx._independent)
#jj.show()

In [ ]:
##### Arrigo, Example 2.19, pp. 49
x =  var("x")
y = function("y")

ode4=diff(y(x),x,x) + 3*y(x)*diff(y(x),x)+y(x)**3
inf = infinitesimalsODE(ode4, y ,x)
print("=========")
for _ in inf:
    _.show()

In [ ]:
# Arrigo, Example 2.20
ode5=diff(y(x),x,x,x) + y(x)*diff(y(x),x,x)
inf = infinitesimalsODE(ode5, y ,x)
print("=========")
for _ in inf:
    _.show()